In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import plotly.graph_objs as go
import numpy as np
from plotly_resampler.figure_resampler import FigureWidgetResampler
from plotly.subplots import make_subplots

In [3]:
n = 1_000_000  # the nbr of data points
x = np.arange(n)
y = np.sin(x / 200) + np.random.random(len(x)) / 10

**note**: 
* to use `FigureWidgetResampler`, you need to have `ipywidgets` installed
* `FigureWidgetResampler` does not start a web applcication, making this wrapper suitable jupyter based-environemnts, where multiple `FigureWidgets` can be created

### Basic `FigureWidgetResampler` example

To utilize FigureWidgetResampler, you just need to:
* wrap your figure with a `FigureWidgetResampler`
* output this wrapped instance in a cell

In [4]:
# Wrap a figure with FigureWidgetResampler
fw_fig = FigureWidgetResampler(make_subplots(rows=2, shared_xaxes=False))

fw_fig.add_trace(go.Scattergl(), hf_x=x, hf_y=y * 1.000003**x, row=1, col=1)
fw_fig.add_trace(go.Scattergl(), hf_x=x, hf_y=(y + 3) * 0.999997**x, row=2, col=1)
fw_fig.update_layout(height=500, showlegend=True)

# Output this wrapped instance in a cell
fw_fig

FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> trace 0 <i style="color:#fc…

Note how the cells below are able to add data to the `FigureWidget`

In [5]:
fw_fig.add_trace(go.Scattergl(), hf_x=x, hf_y=y * 1.0000028**x, row=1, col=1)
fw_fig.add_trace(go.Scattergl(), hf_x=x, hf_y=y * 1.000002**x, row=1, col=1)

In [6]:
fw_fig.add_trace(go.Scattergl(), hf_x=x, hf_y=(y + 3) * 0.999999**x, row=2, col=1)

### time-series **annotation** using the on-click callback

In [7]:
from ipywidgets import Dropdown
from IPython.display import display

In [12]:
# Proxy for a label class
label_dict = {
    "peak": {
        "type": "marker",
        "trace_index": 1,
        "plt_kwargs": {
        "mode": "markers", "marker_symbol": "star", "marker_size": 12, "marker_color": "red"
        }
    },
    "through": {
        "type": "marker",
        "trace_index": 2,
        "plt_kwargs": {
        "mode": "markers", "marker_symbol": "cross", "marker_size": 12, "line_color": "orange"
        }
    },
    "rise": {
        "type": "x-range",
        "plt_kwargs": {
            "line_width": 0, "fillcolor": "green", "opacity": 0.3
        }
    },
    "fall": {
        "type": "x-range",
        "plt_kwargs": {
            "line_width": 0, "fillcolor": "purple", "opacity": 0.3
        }
    }
}

# Create a label selector
label_selector = Dropdown()
label_selector.options = list(label_dict.keys())

# Construct the figure
fw_fig = FigureWidgetResampler()
fw_fig.add_trace(go.Scattergl(name="noisy sine", opacity=0.8), hf_x=x, hf_y=y)
for k, v in label_dict.items():
    if v.get("type", "").lower() == 'marker':
        fw_fig.add_trace(go.Scattergl(name=k, **v.get("plt_kwargs", {})))


# Update logic
prev_x = []
point_list = []

def update_point(trace, points, selector):
    global prev_x, point_list
    config = label_dict[label_selector.value]

    if config.get("type", "") == 'x-range':
        prev_x.append(points.xs[0])
        if len(prev_x) == 2:
            fw_fig.add_vrect(prev_x[0], prev_x[1], **config.get("plt_kwargs", {}))
            prev_x = []

    with fw_fig.batch_update():
        if config.get("type", "") == 'marker':
            trace_index = config.get("trace_index")
            fw_fig.data[trace_index].x = list(fw_fig.data[trace_index].x) + points.xs
            fw_fig.data[trace_index].y = list(fw_fig.data[trace_index].y) + points.ys

fw_fig.data[0].on_click(update_point)

display(label_selector)
fw_fig

Dropdown(options=('peak', 'through', 'rise', 'fall'), value='peak')

FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> noisy sine <i style="color:…

### Adjusting the figure data using the `hf_data` property

In [9]:
fw_fig = FigureWidgetResampler(make_subplots(rows=2, shared_xaxes=False), verbose=True)
fw_fig.update_layout(template='plotly_dark', height=600)
fw_fig.add_trace(go.Scattergl(), hf_x=x, hf_y=y, row=1, col=1)
fw_fig.add_trace(go.Scattergl(), hf_x=x, hf_y=y, row=2, col=1)
fw_fig

	[i] DOWNSAMPLE None	1000000->1000
	[i] DOWNSAMPLE None	1000000->1000


FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> trace 0 <i style="color:#fc…

In [10]:
# After you've ran this cell, reset the axes of the above figure 
# or zoom in-out on the x-range
fw_fig.hf_data[0]['y'] = - (y  + 3) * x / 1000
fw_fig.hf_data[1]['y'] =  (y  + 3) * x / 1000

### Multiple shared traces

In [11]:
fig = FigureWidgetResampler()

# 10 sensors, 500_000 datapoints each on a singe plot
for i in range(10):
    fig.add_trace(go.Scattergl(name=str(i)), hf_x=x, hf_y=y + i)

fig.update_layout(
    height=600, title=f"10 traces -> {10*500_000:,} datapoints", title_x=0.5
)
fig

FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> 0 <i style="color:#fc9944">…